<a href="https://colab.research.google.com/github/Siu0901/AI_study/blob/main/RNN%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EC%83%9D%EC%84%B1_%EA%B3%B5%EB%B6%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [4]:
print(tokenizer.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [6]:
sequences = list()
for line in text.split('\n'):
  encoded = tokenizer.texts_to_sequences([line])[0]
  print(encoded)
  for i in range(1, len(encoded)):
    sequence = encoded[:i+1]
    sequences.append(sequence)

print('학습에 사용할 샘플의 개수 : %d' % len(sequences))

[2, 3, 1, 4, 5]
[]
[6, 1, 7]
[]
[8, 1, 9, 10, 1, 11]
[]
학습에 사용할 샘플의 개수 : 11


In [7]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [8]:
max_len = max(len(l) for l in sequences)
max_len

6

In [9]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [10]:
# 각 샘플의 마지막 단어를 레이블로 분리
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [11]:
print(X)
print(y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
[ 3  1  4  5  1  7  1  9 10  1 11]


In [12]:
y = to_categorical(y, num_classes=vocab_size)

In [13]:
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [ ]:
# 모델 설계하기

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [15]:
# 임베딩 벡터 차원 10
embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 2s - 2s/step - accuracy: 0.1818 - loss: 2.4621
Epoch 2/200
1/1 - 0s - 46ms/step - accuracy: 0.1818 - loss: 2.4495
Epoch 3/200
1/1 - 0s - 45ms/step - accuracy: 0.1818 - loss: 2.4366
Epoch 4/200
1/1 - 0s - 44ms/step - accuracy: 0.1818 - loss: 2.4233
Epoch 5/200
1/1 - 0s - 44ms/step - accuracy: 0.0909 - loss: 2.4095
Epoch 6/200
1/1 - 0s - 55ms/step - accuracy: 0.1818 - loss: 2.3952
Epoch 7/200
1/1 - 0s - 52ms/step - accuracy: 0.1818 - loss: 2.3803
Epoch 8/200
1/1 - 0s - 44ms/step - accuracy: 0.1818 - loss: 2.3646
Epoch 9/200
1/1 - 0s - 46ms/step - accuracy: 0.1818 - loss: 2.3483
Epoch 10/200
1/1 - 0s - 44ms/step - accuracy: 0.1818 - loss: 2.3311
Epoch 11/200
1/1 - 0s - 44ms/step - accuracy: 0.1818 - loss: 2.3130
Epoch 12/200
1/1 - 0s - 50ms/step - accuracy: 0.1818 - loss: 2.2940
Epoch 13/200
1/1 - 0s - 46ms/step - accuracy: 0.2727 - loss: 2.2741
Epoch 14/200
1/1 - 0s - 45ms/step - accuracy: 0.3636 - loss: 2.2533
Epoch 15/200
1/1 - 0s - 44ms/step - accuracy: 0.4545 - loss

In [28]:
def sentence_generation(model, tokenizer, current_word, n):
  init_word = current_word
  sentence = ''

  # n번 반복
  for _ in range(n):
    # 현재 단어에 대한 정수 인코딩과 패딩
    encoded = tokenizer.texts_to_sequences([current_word])[0]
    encoded = pad_sequences([encoded], maxlen=5, padding='pre')
    print(encoded)

    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
    result = model.predict(encoded, verbose=0)
    result = np.argmax(result, axis=1)

    for word, index in tokenizer.word_index.items():
        # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
        if index == result:
            break
    # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
    current_word = current_word + ' '  + word
    print(current_word)
    # 예측 단어를 문장에 저장
    sentence = sentence + ' ' + word
    print(sentence)

  sentence = init_word + sentence
  return sentence

In [29]:
print(sentence_generation(model, tokenizer, '경마장에', 4))

[[0 0 0 0 2]]
경마장에 있는
 있는
[[0 0 0 2 3]]
경마장에 있는 말이
 있는 말이
[[0 0 2 3 1]]
경마장에 있는 말이 뛰고
 있는 말이 뛰고
[[0 2 3 1 4]]
경마장에 있는 말이 뛰고 있다
 있는 말이 뛰고 있다
경마장에 있는 말이 뛰고 있다


In [30]:
print(sentence_generation(model, tokenizer, '그의', 2))

[[0 0 0 0 6]]
그의 말이
 말이
[[0 0 0 6 1]]
그의 말이 법이다
 말이 법이다
그의 말이 법이다


In [31]:
print(sentence_generation(model, tokenizer, '가는', 5))

[[0 0 0 0 8]]
가는 말이
 말이
[[0 0 0 8 1]]
가는 말이 고와야
 말이 고와야
[[0 0 8 1 9]]
가는 말이 고와야 오는
 말이 고와야 오는
[[ 0  8  1  9 10]]
가는 말이 고와야 오는 말이
 말이 고와야 오는 말이
[[ 8  1  9 10  1]]
가는 말이 고와야 오는 말이 곱다
 말이 고와야 오는 말이 곱다
가는 말이 고와야 오는 말이 곱다
